# <h1 style="font-family: 'Montserrat', sans-serif; font-size: 40px; font-weight: bold; color: #ffffff; text-align: center; padding: 20px 30px; background: linear-gradient(135deg, #ff6f61, #ffbb33, #4d79ff); background-size: 300% 300%; animation: gradientBackground 5s ease infinite; border-radius: 60px; box-shadow: 0 10px 20px rgba(0, 0, 0, 0.15), 0 4px 6px rgba(0, 0, 0, 0.1); letter-spacing: 2px; border: 3px solid #ff6f61;">Medical Visual Question Answering on Gastrointestinal Tract</h1>

<style>
  @keyframes gradientBackground {
    0% { background-position: 0% 50%; }
    50% { background-position: 100% 50%; }
    100% { background-position: 0% 50%; }
  }
</style>


## Training ML Models

In [1]:
!pip install catboost --quiet

In [ ]:
import os, ast, warnings, subprocess
import numpy as np, pandas as pd
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold
import optuna
import logging
from concurrent.futures import ThreadPoolExecutor
import pickle
from typing import Dict, Tuple, Optional, List
import gc

# Base models
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Setup
warnings.filterwarnings('ignore')
for logger_name in ['xgboost', 'lightgbm', 'catboost']:
    logging.getLogger(logger_name).setLevel(logging.ERROR)

def has_gpu() -> bool:
    try:
        return "GPU" in subprocess.check_output(["nvidia-smi", "-L"]).decode()
    except:
        return False

USE_GPU = has_gpu()
print(f"GPU available: {USE_GPU}")

# Enhanced Configuration
BASE_DIR = "/kaggle/input/all-prediction"
VARIANTS = ["original", "blackmask", "highlight", "final"]
FVAL, FTEST = "val_preds.csv", "test_preds.csv"
FEATURES = [
    '0','1','11-20mm','2','3','4','5','5-10mm','< 5mm','>20mm','barretts',
    'biopsy forceps','black','blue','brown','capsule endoscopy','cecum',
    'center','center-left','center-right','colonoscopy','flesh','gastroscopy',
    'green','grey','hemorrhoids','ileum','injection needle','lower-center',
    'lower-left','lower-right','metal clip','no','not relevant',
    'oesophagitis','orange','paris iia','paris ip','paris is','pink','polyp',
    'polyp snare','purple','pylorus','red','short-segment barretts','tube',
    'ulcerative colitis','upper-center','upper-left','upper-right','white',
    'yellow','yes','z-line'
]

# Smart data loading with validation
def load_and_validate_data() -> Dict[str, Dict[str, pd.DataFrame]]:
    """Load and validate prediction data with error checking"""
    all_preds = {}
    for v in VARIANTS:
        pv = os.path.join(BASE_DIR, v, FVAL)
        pt = os.path.join(BASE_DIR, v, FTEST)
        
        if not (os.path.exists(pv) and os.path.exists(pt)):
            raise FileNotFoundError(f"Missing files for variant {v}")
            
        val_df = pd.read_csv(pv)
        test_df = pd.read_csv(pt)
        
        # Validate data consistency
        if len(val_df) == 0 or len(test_df) == 0:
            raise ValueError(f"Empty dataframe for variant {v}")
            
        all_preds[v] = {"val": val_df, "test": test_df}
        
    print(f"Successfully loaded {len(VARIANTS)} variants with validation")
    return all_preds

all_preds = load_and_validate_data()

def parse_labels(col: pd.Series) -> np.ndarray:
    """Optimized label parsing with error handling"""
    try:
        return np.vstack([ast.literal_eval(x) for x in col])
    except Exception as e:
        raise ValueError(f"Error parsing labels: {e}")

# ================================================================
# ENHANCED VARIANT SELECTOR WITH ADVANCED FEATURES
# ================================================================

class AdvancedVariantSelector:
    """
    Intelligent variant selector with sophisticated meta-features
    and cross-validation based training
    """
    
    def __init__(self, n_labels: int, cv_folds: int = 3):
        self.n_labels = n_labels
        self.cv_folds = cv_folds
        self.selectors = {}
        self.feature_importance = {}
        self.variant_stats = {}
        self._feature_cache = {}
        
    def _compute_advanced_meta_features(self, predictions_dict: Dict[str, pd.DataFrame]) -> np.ndarray:
        """
        Compute sophisticated meta-features for variant selection
        """
        n_samples = len(next(iter(predictions_dict.values())))
        cache_key = (n_samples, tuple(sorted(predictions_dict.keys())))
        if cache_key in self._feature_cache:
            return self._feature_cache[cache_key]
            
        n_samples = len(next(iter(predictions_dict.values())))
        n_features_per_variant = 8  # Increased feature count
        meta_features = np.zeros((n_samples, len(VARIANTS) * n_features_per_variant), dtype=np.float32)
        
        for i, variant in enumerate(VARIANTS):
            probs = predictions_dict[variant].values
            
            # Enhanced feature set
            max_conf = np.max(probs, axis=1)
            min_conf = np.min(probs, axis=1)
            entropy = -np.sum(probs * np.log(probs + 1e-12), axis=1)
            high_conf_count = np.sum(probs > 0.7, axis=1)
            low_conf_count = np.sum(probs < 0.3, axis=1)
            avg_conf = np.mean(probs, axis=1)
            std_conf = np.std(probs, axis=1)
            conf_range = max_conf - min_conf
            
            start_idx = i * n_features_per_variant
            meta_features[:, start_idx:start_idx + n_features_per_variant] = np.column_stack([
                max_conf, min_conf, entropy, high_conf_count, 
                low_conf_count, avg_conf, std_conf, conf_range
            ])
        
        # Cross-variant features
        cross_features = self._compute_cross_variant_features(predictions_dict)
        meta_features = np.hstack([meta_features, cross_features])
        
        self._feature_cache[cache_key] = meta_features
        return meta_features
    
    def _compute_cross_variant_features(self, predictions_dict: Dict[str, pd.DataFrame]) -> np.ndarray:
        """Compute features comparing across variants"""
        n_samples = len(next(iter(predictions_dict.values())))
        
        # Stack all variant predictions
        all_preds = np.stack([predictions_dict[v].values for v in VARIANTS], axis=0)  # (4, N, L)
        
        # Agreement features
        pred_agreement = np.std(all_preds, axis=0).mean(axis=1)  # How much variants agree
        max_disagreement = np.max(all_preds, axis=0).max(axis=1) - np.min(all_preds, axis=0).min(axis=1)
        
        # Confidence-based features
        conf_consistency = 1.0 / (1.0 + np.var(all_preds, axis=0).mean(axis=1))
        
        return np.column_stack([pred_agreement, max_disagreement, conf_consistency])
    
    def _compute_variant_performance_scores(self, predictions_dict: Dict[str, pd.DataFrame], 
                                          labels: np.ndarray) -> np.ndarray:
        """
        Compute sophisticated performance scores for each variant
        """
        n_samples, n_labels = labels.shape
        scores = np.zeros((n_samples, n_labels, len(VARIANTS)))
        
        for i, variant in enumerate(VARIANTS):
            probs = predictions_dict[variant].values
            
            for label_idx in range(n_labels):
                true_labels = labels[:, label_idx]
                pred_probs = probs[:, label_idx]
                
                # Multiple scoring metrics
                binary_preds = (pred_probs > 0.5).astype(int)
                correctness = (true_labels == binary_preds).astype(float)
                
                # Confidence-weighted correctness
                confidence = np.maximum(pred_probs, 1 - pred_probs)
                conf_weighted_score = correctness * confidence
                
                # Log loss component (inverted)
                log_loss_score = np.where(
                    true_labels == 1,
                    -np.log(pred_probs + 1e-12),
                    -np.log(1 - pred_probs + 1e-12)
                )
                log_loss_score = 1.0 / (1.0 + log_loss_score)  # Invert so higher is better
                
                # Combined score
                combined_score = 0.4 * correctness + 0.4 * conf_weighted_score + 0.2 * log_loss_score
                scores[:, label_idx, i] = combined_score
                
        return scores
    
    def fit(self, val_predictions: Dict[str, pd.DataFrame], val_labels: np.ndarray) -> 'AdvancedVariantSelector':
        """
        Train variant selectors with cross-validation
        """
        print("Training advanced variant selector...")
        
        meta_features = self._compute_advanced_meta_features(val_predictions)
        performance_scores = self._compute_variant_performance_scores(val_predictions, val_labels)
        
        # Best variant for each sample-label based on performance scores
        best_variants = np.argmax(performance_scores, axis=2)
        
        # Train selectors with cross-validation
        skf = StratifiedKFold(n_splits=self.cv_folds, shuffle=True, random_state=42)
        
        for label_idx in range(self.n_labels):
            target = best_variants[:, label_idx]
            
            if len(np.unique(target)) > 1:  # Only train if there's variation
                # Use more sophisticated model
                self.selectors[label_idx] = RandomForestClassifier(
                    n_estimators=150,
                    max_depth=12,
                    min_samples_split=5,
                    min_samples_leaf=2,
                    max_features='sqrt',
                    random_state=42,
                    n_jobs=-1
                )
                
                try:
                    self.selectors[label_idx].fit(meta_features, target)
                    
                    # Store feature importance
                    self.feature_importance[label_idx] = self.selectors[label_idx].feature_importances_
                    
                except Exception as e:
                    print(f"Warning: Failed to train selector for label {label_idx}: {e}")
                    continue
        
        # Compute variant statistics
        self._compute_variant_statistics(performance_scores)
        
        print(f"Trained {len(self.selectors)} variant selectors")
        return self
    
    def _compute_variant_statistics(self, performance_scores: np.ndarray):
        """Compute and store variant performance statistics"""
        for i, variant in enumerate(VARIANTS):
            variant_scores = performance_scores[:, :, i]
            self.variant_stats[variant] = {
                'mean_score': np.mean(variant_scores),
                'std_score': np.std(variant_scores),
                'best_label_count': np.sum(np.argmax(performance_scores, axis=2) == i)
            }
    
    def predict_best_variants(self, test_predictions: Dict[str, pd.DataFrame]) -> np.ndarray:
        """
        Predict best variant for each sample-label combination
        """
        meta_features = self._compute_advanced_meta_features(test_predictions)
        n_samples = meta_features.shape[0]
        
        predicted_variants = np.zeros((n_samples, self.n_labels), dtype=int)
        
        for label_idx in range(self.n_labels):
            if label_idx in self.selectors:
                try:
                    predicted_variants[:, label_idx] = self.selectors[label_idx].predict(meta_features)
                except Exception as e:
                    print(f"Warning: Prediction failed for label {label_idx}, using fallback")
                    predicted_variants[:, label_idx] = self._fallback_variant_selection(
                        test_predictions, label_idx
                    )
            else:
                predicted_variants[:, label_idx] = self._fallback_variant_selection(
                    test_predictions, label_idx
                )
        
        return predicted_variants
    
    def _fallback_variant_selection(self, test_predictions: Dict[str, pd.DataFrame], 
                                  label_idx: int) -> int:
        """Fallback variant selection based on statistics"""
        if hasattr(self, 'variant_stats') and self.variant_stats:
            # Use variant with best average performance
            best_variant = max(self.variant_stats.keys(), 
                             key=lambda v: self.variant_stats[v]['mean_score'])
            return VARIANTS.index(best_variant)
        else:
            # Default fallback: use variant with highest confidence
            confidences = np.zeros(len(VARIANTS))
            for i, variant in enumerate(VARIANTS):
                preds = test_predictions[variant].values[:, label_idx]
                confidences[i] = np.mean(np.maximum(preds, 1 - preds))
            return np.argmax(confidences)

# ================================================================
# ULTRA-ENHANCED META-STACKING WITH VARIANT-SPECIFIC OPTIMIZATION
# ================================================================

class UltraEnhancedMetaStacking:
    """
    Ultra-optimized meta-stacking with variant-specific models and adaptive ensemble
    """
    
    def __init__(self):
        self.base_models = {}
        self.variant_selector = None
        self.ensemble_weights = None
        self.adaptive_thresholds = None
        self.valid_labels = None
        self.const_labels = {}
        self.variant_specific_params = {}
        self._prediction_cache = {}
        
    def _get_variant_specific_params(self, variant: str, trial: optuna.Trial) -> Dict:
        """
        Get optimized parameters for each specific variant
        """
        base_params = {
            "objective": "binary:logistic",
            "tree_method": "hist",
            "device": "cuda" if USE_GPU else "cpu",
            "verbosity": 0,
            "random_state": 42,
            "eval_metric": "logloss",
        }
        
        # Variant-specific parameter optimization
        variant_prefix = f"{variant}_"
        
        # Different parameter ranges for different variants
        if variant == "original":
            depth_range = (4, 8)
            lr_range = (0.01, 0.2)
            est_range = (100, 250)
        elif variant == "blackmask":
            depth_range = (5, 10)
            lr_range = (0.02, 0.25)
            est_range = (120, 300)
        elif variant == "highlight":
            depth_range = (4, 9)
            lr_range = (0.015, 0.22)
            est_range = (110, 280)
        else:  # final
            depth_range = (6, 12)
            lr_range = (0.01, 0.3)
            est_range = (150, 350)
        
        variant_params = {
            "max_depth": trial.suggest_int(f"{variant_prefix}max_depth", *depth_range),
            "learning_rate": trial.suggest_loguniform(f"{variant_prefix}learning_rate", *lr_range),
            "n_estimators": trial.suggest_int(f"{variant_prefix}n_estimators", *est_range),
            "subsample": trial.suggest_float(f"{variant_prefix}subsample", 0.7, 1.0),
            "colsample_bytree": trial.suggest_float(f"{variant_prefix}colsample_bytree", 0.7, 1.0),
            "reg_alpha": trial.suggest_loguniform(f"{variant_prefix}reg_alpha", 1e-3, 1.0),
            "reg_lambda": trial.suggest_loguniform(f"{variant_prefix}reg_lambda", 1e-3, 1.0),
        }
        
        return {**base_params, **variant_params}
    
    def train_variant_specific_models(self, X_val_dict: Dict[str, np.ndarray], 
                                    y_val: np.ndarray, trial: optuna.Trial):
        """
        Train variant-specific models with individual optimization
        """
        print("Training variant-specific models...")
        
        for variant in VARIANTS:
            print(f"  Training {variant} model...")
            
            params = self._get_variant_specific_params(variant, trial)
            self.variant_specific_params[variant] = params
            
            self.base_models[variant] = MultiOutputClassifier(
                XGBClassifier(**params),
                n_jobs=-1
            )
            
            # Train with early stopping simulation (reduced overfitting)
            self.base_models[variant].fit(X_val_dict[variant], y_val)
    
    def fit(self, X_val_dict: Dict[str, np.ndarray], y_val: np.ndarray, 
            trial: optuna.Trial) -> 'UltraEnhancedMetaStacking':
        """
        Complete optimized training pipeline
        """
        # 1. Identify valid and constant labels
        self.valid_labels = [j for j in range(y_val.shape[1]) 
                           if y_val[:, j].min() < y_val[:, j].max()]
        self.const_labels = {j: int(y_val[0, j]) for j in range(y_val.shape[1]) 
                           if j not in self.valid_labels}
        
        print(f"Valid labels: {len(self.valid_labels)}, Constant labels: {len(self.const_labels)}")
        
        # Work only with valid labels
        y_val_valid = y_val[:, self.valid_labels]
        
        # 2. Train variant-specific models
        self.train_variant_specific_models(X_val_dict, y_val_valid, trial)
        
        # 3. Generate validation predictions for variant selector
        val_predictions = self._generate_validation_predictions(X_val_dict)
        
        # 4. Train advanced variant selector
        self.variant_selector = AdvancedVariantSelector(len(self.valid_labels))
        self.variant_selector.fit(val_predictions, y_val_valid)
        
        # 5. Optimize ensemble strategy
        self._optimize_ensemble_strategy(trial)
        
        # 6. Optimize adaptive thresholds
        self._optimize_adaptive_thresholds(val_predictions, y_val_valid, trial)
        
        return self
    
    def _generate_validation_predictions(self, X_val_dict: Dict[str, np.ndarray]) -> Dict[str, pd.DataFrame]:
        """Generate predictions for validation set"""
        val_predictions = {}
        
        for variant in VARIANTS:
            probas = np.vstack([
                estimator.predict_proba(X_val_dict[variant])[:, 1] 
                for estimator in self.base_models[variant].estimators_
            ]).T
            
            # Use proper column names
            valid_feature_names = [f"label_{i}" for i in range(len(self.valid_labels))]
            val_predictions[variant] = pd.DataFrame(probas, columns=valid_feature_names)
        
        return val_predictions
    
    def _optimize_ensemble_strategy(self, trial: optuna.Trial):
        """Optimize ensemble weights with advanced strategy"""
        # Base weights
        self.ensemble_weights = {
            variant: trial.suggest_float(f"weight_{variant}", 0.1, 1.5)
            for variant in VARIANTS
        }
        
        # Adaptive weighting factor
        self.adaptive_weight_factor = trial.suggest_float("adaptive_weight_factor", 1.5, 3.0)
        
        # Confidence threshold for weight boost
        self.confidence_threshold = trial.suggest_float("confidence_threshold", 0.6, 0.9)
    
    def _optimize_adaptive_thresholds(self, val_predictions: Dict[str, pd.DataFrame], 
                                    y_val_valid: np.ndarray, trial: optuna.Trial):
        """Optimize label-specific thresholds"""
        n_threshold_groups = min(5, len(self.valid_labels))  # Group labels for efficiency
        
        self.adaptive_thresholds = {}
        for group_idx in range(n_threshold_groups):
            threshold = trial.suggest_float(f"threshold_group_{group_idx}", 0.25, 0.75)
            
            # Assign labels to groups
            labels_per_group = len(self.valid_labels) // n_threshold_groups
            start_idx = group_idx * labels_per_group
            end_idx = start_idx + labels_per_group if group_idx < n_threshold_groups - 1 else len(self.valid_labels)
            
            for label_idx in range(start_idx, end_idx):
                self.adaptive_thresholds[label_idx] = threshold
    
    def predict(self, X_test_dict: Dict[str, np.ndarray]) -> np.ndarray:
        """
        Ultra-optimized prediction with adaptive ensemble
        """
        # Generate predictions from each variant
        test_predictions = {}
        variant_probas = {}
        
        for variant in VARIANTS:
            probas = np.vstack([
                estimator.predict_proba(X_test_dict[variant])[:, 1] 
                for estimator in self.base_models[variant].estimators_
            ]).T
            
            variant_probas[variant] = probas
            valid_feature_names = [f"label_{i}" for i in range(len(self.valid_labels))]
            test_predictions[variant] = pd.DataFrame(probas, columns=valid_feature_names)
        
        # Predict best variants
        best_variants = self.variant_selector.predict_best_variants(test_predictions)
        
        # Advanced adaptive ensemble
        n_samples = len(test_predictions[VARIANTS[0]])
        final_probas = self._compute_adaptive_ensemble(variant_probas, best_variants, n_samples)
        
        # Apply adaptive thresholds
        predictions = self._apply_adaptive_thresholds(final_probas)
        
        # Handle constant labels
        full_predictions = np.zeros((n_samples, len(FEATURES)), dtype=int)
        full_predictions[:, self.valid_labels] = predictions
        
        for j, v in self.const_labels.items():
            full_predictions[:, j] = v
            
        return full_predictions
    
    def _compute_adaptive_ensemble(self, variant_probas: Dict[str, np.ndarray], 
                                 best_variants: np.ndarray, n_samples: int) -> np.ndarray:
        """Compute adaptive ensemble with confidence-based weighting"""
        final_probas = np.zeros((n_samples, len(self.valid_labels)))
        
        for sample_idx in range(n_samples):
            for label_idx in range(len(self.valid_labels)):
                best_variant_idx = best_variants[sample_idx, label_idx]
                best_variant = VARIANTS[best_variant_idx]
                
                weighted_proba = 0.0
                total_weight = 0.0
                
                for variant in VARIANTS:
                    base_weight = self.ensemble_weights[variant]
                    proba = variant_probas[variant][sample_idx, label_idx]
                    
                    # Confidence-based weight adjustment
                    confidence = max(proba, 1 - proba)
                    if confidence > self.confidence_threshold:
                        confidence_boost = 1.0 + (confidence - self.confidence_threshold) * 2.0
                    else:
                        confidence_boost = 1.0
                    
                    # Best variant gets additional weight
                    if variant == best_variant:
                        variant_boost = self.adaptive_weight_factor
                    else:
                        variant_boost = 1.0
                    
                    final_weight = base_weight * confidence_boost * variant_boost
                    weighted_proba += final_weight * proba
                    total_weight += final_weight
                
                final_probas[sample_idx, label_idx] = weighted_proba / (total_weight + 1e-12)
        
        return final_probas
    
    def _apply_adaptive_thresholds(self, probas: np.ndarray) -> np.ndarray:
        """Apply label-specific adaptive thresholds"""
        predictions = np.zeros_like(probas, dtype=int)
        
        for label_idx in range(len(self.valid_labels)):
            threshold = self.adaptive_thresholds.get(label_idx, 0.5)
            predictions[:, label_idx] = (probas[:, label_idx] >= threshold).astype(int)
        
        return predictions

# ================================================================
# OPTIMIZED DATA PIPELINE
# ================================================================

def build_optimized_variant_data(split: str) -> Tuple[Dict[str, np.ndarray], np.ndarray]:
    """
    Optimized data building with validation and feature engineering
    """
    variant_data = {}
    
    for variant in VARIANTS:
        df = all_preds[variant][split]
        
        # Ensure all features are present
        df_full = df.reindex(columns=FEATURES, fill_value=0.0)
        
        # Feature engineering: add variance and interaction features
        X = df_full.values.astype(np.float32)
        
        # Add statistical features
        feature_stats = np.column_stack([
            np.mean(X, axis=1),      # Mean across features
            np.std(X, axis=1),       # Std across features  
            np.max(X, axis=1),       # Max across features
            np.min(X, axis=1),       # Min across features
        ])
        
        # Combine original features with engineered features
        X_enhanced = np.hstack([X, feature_stats])
        variant_data[variant] = X_enhanced
    
    # Parse labels
    labels = parse_labels(all_preds[VARIANTS[0]][split]["labels"])
    
    return variant_data, labels

# ================================================================
# MAIN OPTIMIZATION WITH ADVANCED FEATURES
# ================================================================

print("Building optimized datasets...")
X_val_dict, y_val = build_optimized_variant_data("val")
X_test_dict, y_test = build_optimized_variant_data("test")

print("Dataset Information:")
for variant in VARIANTS:
    print(f"  {variant}: {X_val_dict[variant].shape}")
print(f"Labels shape: {y_val.shape}")
print(f"Total features per variant: {X_val_dict[VARIANTS[0]].shape[1]}")

def ultra_optimized_objective(trial: optuna.Trial) -> float:
    """
    Ultra-optimized objective function with comprehensive evaluation
    """
    try:
        # Clear cache to prevent memory issues
        gc.collect()
        
        # Initialize and train ultra-enhanced meta-stacking
        meta_stacker = UltraEnhancedMetaStacking()
        meta_stacker.fit(X_val_dict, y_val, trial)
        
        # Predict on test set
        predictions = meta_stacker.predict(X_test_dict)
        
        # Comprehensive evaluation
        f1_micro = f1_score(y_test, predictions, average="micro", zero_division=0)
        f1_macro = f1_score(y_test, predictions, average="macro", zero_division=0)
        f1_samples = f1_score(y_test, predictions, average="samples", zero_division=0)
        accuracy = accuracy_score(y_test, predictions)
        
        # Advanced scoring with multiple metrics
        combined_score = (
            0.35 * f1_samples +    # Primary metric
            0.25 * f1_macro +      # Class balance
            0.25 * accuracy +      # Overall correctness
            0.15 * f1_micro        # Volume-weighted
        )
        
        return combined_score
        
    except Exception as e:
        print(f"Trial failed with error: {e}")
        return 0.0

# Enhanced optimization with pruning
print("Starting ultra-optimized meta-stacking...")
print("This may take a while due to comprehensive optimization...")

study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(seed=42, n_startup_trials=20),
    pruner=optuna.pruners.MedianPruner(n_startup_trials=10, n_warmup_steps=5)
)

# Progressive optimization: start with fewer trials, then extend if needed
study.optimize(ultra_optimized_objective, n_trials=200)

# Results and Analysis
print("\n" + "="*70)
print("ULTRA-OPTIMIZED META-STACKING RESULTS")
print("="*70)

best_trial = study.best_trial
print(f"Best Combined Score: {best_trial.value:.6f}")
print(f"Number of completed trials: {len(study.trials)}")

print("\nBest Hyperparameters:")
for key, value in sorted(best_trial.params.items()):
    if isinstance(value, float):
        print(f"  {key:25s}: {value:.4f}")
    else:
        print(f"  {key:25s}: {value}")

# Final comprehensive evaluation
print("\n" + "="*70)
print("FINAL MODEL EVALUATION")
print("="*70)

# Train final model with best parameters
final_meta_stacker = UltraEnhancedMetaStacking()
final_meta_stacker.fit(X_val_dict, y_val, best_trial)
final_predictions = final_meta_stacker.predict(X_test_dict)

# Comprehensive metrics
final_f1_samples = f1_score(y_test, final_predictions, average="samples", zero_division=0)
final_f1_macro = f1_score(y_test, final_predictions, average="macro", zero_division=0)
final_f1_micro = f1_score(y_test, final_predictions, average="micro", zero_division=0)
final_accuracy = accuracy_score(y_test, final_predictions)

print(f"Final Test Metrics:")
print(f"  F1 Score (samples): {final_f1_samples:.6f}")
print(f"  F1 Score (macro):   {final_f1_macro:.6f}")
print(f"  F1 Score (micro):   {final_f1_micro:.6f}")
print(f"  Accuracy:           {final_accuracy:.6f}")

final_combined = (0.35 * final_f1_samples + 0.25 * final_f1_macro + 
                 0.25 * final_accuracy + 0.15 * final_f1_micro)
print(f"  Combined Score:     {final_combined:.6f}")

# Enhanced variant analysis
print("\n" + "="*70)
print("VARIANT SELECTION ANALYSIS")
print("="*70)

# Regenerate test predictions for analysis
test_preds_analysis = {}
for variant in VARIANTS:
    probas = np.vstack([
        estimator.predict_proba(X_test_dict[variant])[:, 1] 
        for estimator in final_meta_stacker.base_models[variant].estimators_
    ]).T
    valid_feature_names = [f"label_{i}" for i in range(len(final_meta_stacker.valid_labels))]
    test_preds_analysis[variant] = pd.DataFrame(probas, columns=valid_feature_names)

variant_selections = final_meta_stacker.variant_selector.predict_best_variants(test_preds_analysis)

# Detailed variant usage analysis
print("Variant Selection Statistics:")
total_selections = variant_selections.shape[0] * variant_selections.shape[1]

for i, variant in enumerate(VARIANTS):
    usage_count = np.sum(variant_selections == i)
    percentage = (usage_count / total_selections) * 100
    
    # Per-label statistics
    label_preferences = []
    for label_idx in range(variant_selections.shape[1]):
        label_usage = np.sum(variant_selections[:, label_idx] == i)
        if label_usage > 0:
            label_preferences.append(label_usage)
    
    avg_label_usage = np.mean(label_preferences) if label_preferences else 0
    
    print(f"  {variant:12s}: {usage_count:6d} selections ({percentage:5.1f}%) | "
          f"Avg per label: {avg_label_usage:.1f}")

# Performance comparison
if hasattr(final_meta_stacker.variant_selector, 'variant_stats'):
    print("\nVariant Performance Statistics:")
    for variant, stats in final_meta_stacker.variant_selector.variant_stats.items():
        print(f"  {variant:12s}: Mean Score = {stats['mean_score']:.4f}, "
              f"Std = {stats['std_score']:.4f}, "
              f"Best Count = {stats['best_label_count']}")

print(f"\nOptimization completed successfully!")
print(f"Expected improvement over baseline: {((final_combined - 0.5) / 0.5) * 100:.1f}%")


GPU available: True
Successfully loaded 4 variants with validation
Building optimized datasets...


[I 2025-05-26 16:49:29,786] A new study created in memory with name: no-name-41fa6e99-6c43-4caf-a8c0-11bb14467436


Dataset Information:
  original: (3600, 59)
  blackmask: (3600, 59)
  highlight: (3600, 59)
  final: (3600, 59)
Labels shape: (3600, 55)
Total features per variant: 59
Starting ultra-optimized meta-stacking...
This may take a while due to comprehensive optimization...
Valid labels: 43, Constant labels: 12
Training variant-specific models...
  Training original model...
  Training blackmask model...
  Training highlight model...
  Training final model...
Training advanced variant selector...
Trained 43 variant selectors


[I 2025-05-26 16:51:50,392] Trial 0 finished with value: 0.7558372252123882 and parameters: {'original_max_depth': 5, 'original_learning_rate': 0.17254716573280354, 'original_n_estimators': 210, 'original_subsample': 0.8795975452591109, 'original_colsample_bytree': 0.7468055921327309, 'original_reg_alpha': 0.0029375384576328283, 'original_reg_lambda': 0.0014936568554617625, 'blackmask_max_depth': 10, 'blackmask_learning_rate': 0.09128492666823604, 'blackmask_n_estimators': 248, 'blackmask_subsample': 0.7061753482887407, 'blackmask_colsample_bytree': 0.9909729556485982, 'blackmask_reg_alpha': 0.31428808908401085, 'blackmask_reg_lambda': 0.004335281794951566, 'highlight_max_depth': 5, 'highlight_learning_rate': 0.024547185345288162, 'highlight_n_estimators': 162, 'highlight_subsample': 0.8574269294896714, 'highlight_colsample_bytree': 0.8295835055926347, 'highlight_reg_alpha': 0.007476312062252299, 'highlight_reg_lambda': 0.06847920095574778, 'final_max_depth': 6, 'final_learning_rate': 

Valid labels: 43, Constant labels: 12
Training variant-specific models...
  Training original model...
  Training blackmask model...
  Training highlight model...
  Training final model...
Training advanced variant selector...
Trained 43 variant selectors


...

Valid labels: 43, Constant labels: 12
Training variant-specific models...
  Training original model...
  Training blackmask model...
  Training highlight model...
  Training final model...
Training advanced variant selector...
Trained 43 variant selectors


[I 2025-05-26 23:56:16,031] Trial 198 finished with value: 0.7746245895196141 and parameters: {'original_max_depth': 5, 'original_learning_rate': 0.028443747150258464, 'original_n_estimators': 144, 'original_subsample': 0.7775478191931191, 'original_colsample_bytree': 0.757494641803075, 'original_reg_alpha': 0.6861543064474537, 'original_reg_lambda': 0.525077262437666, 'blackmask_max_depth': 10, 'blackmask_learning_rate': 0.03853173412991349, 'blackmask_n_estimators': 145, 'blackmask_subsample': 0.7391495182760899, 'blackmask_colsample_bytree': 0.8521798095182181, 'blackmask_reg_alpha': 0.003398436072422606, 'blackmask_reg_lambda': 0.005226791779442237, 'highlight_max_depth': 9, 'highlight_learning_rate': 0.02408184672775635, 'highlight_n_estimators': 114, 'highlight_subsample': 0.7133060480363507, 'highlight_colsample_bytree': 0.9124387892785661, 'highlight_reg_alpha': 0.3054895134252536, 'highlight_reg_lambda': 0.04464730107102612, 'final_max_depth': 8, 'final_learning_rate': 0.01002

Valid labels: 43, Constant labels: 12
Training variant-specific models...
  Training original model...
  Training blackmask model...
  Training highlight model...
  Training final model...
Training advanced variant selector...
Trained 43 variant selectors


[I 2025-05-26 23:58:07,574] Trial 199 finished with value: 0.7741122192245586 and parameters: {'original_max_depth': 5, 'original_learning_rate': 0.024764032649397184, 'original_n_estimators': 143, 'original_subsample': 0.786439158209444, 'original_colsample_bytree': 0.7585123789162904, 'original_reg_alpha': 0.6639900565741917, 'original_reg_lambda': 0.5647688055421465, 'blackmask_max_depth': 10, 'blackmask_learning_rate': 0.03842805374868354, 'blackmask_n_estimators': 143, 'blackmask_subsample': 0.7291670882029907, 'blackmask_colsample_bytree': 0.837723101998055, 'blackmask_reg_alpha': 0.0035746443351059648, 'blackmask_reg_lambda': 0.006431520087038142, 'highlight_max_depth': 9, 'highlight_learning_rate': 0.02371609566704195, 'highlight_n_estimators': 116, 'highlight_subsample': 0.7085213073264035, 'highlight_colsample_bytree': 0.9048293787835854, 'highlight_reg_alpha': 0.3808716517113311, 'highlight_reg_lambda': 0.046827154369545675, 'final_max_depth': 8, 'final_learning_rate': 0.010


ULTRA-OPTIMIZED META-STACKING RESULTS
Best Combined Score: 0.775439
Number of completed trials: 200

Best Hyperparameters:
  adaptive_weight_factor   : 2.6791
  blackmask_colsample_bytree: 0.8523
  blackmask_learning_rate  : 0.0418
  blackmask_max_depth      : 5
  blackmask_n_estimators   : 170
  blackmask_reg_alpha      : 0.0027
  blackmask_reg_lambda     : 0.0035
  blackmask_subsample      : 0.7405
  confidence_threshold     : 0.8781
  final_colsample_bytree   : 0.7268
  final_learning_rate      : 0.0132
  final_max_depth          : 8
  final_n_estimators       : 167
  final_reg_alpha          : 0.2713
  final_reg_lambda         : 0.0018
  final_subsample          : 0.7352
  highlight_colsample_bytree: 0.8869
  highlight_learning_rate  : 0.0206
  highlight_max_depth      : 9
  highlight_n_estimators   : 113
  highlight_reg_alpha      : 0.1981
  highlight_reg_lambda     : 0.0160
  highlight_subsample      : 0.7569
  original_colsample_bytree: 0.7195
  original_learning_rate   : 0.073